 Note that I had an earlier generation of the code already, this earliest generation is called _Bing 1st autonomous_.

 ---
The code below trains the neural network for 100 epochs and prints the results for each epoch. The results include the average loss for the training dataset, and the L1 and L_inf norms of the errors for both the training and test datasets. The code also adjusts the learning rate based on the improvement of the training loss over a patience period and a threshold. The code uses PyTorch’s built-in modules and functions to define and train the neural network, such as nn.Module, nn.Linear, nn.Sigmoid, nn.ReLU, nn.MSELoss, optim.Adam, optim.lr_scheduler.ReduceLROnPlateau, torch.utils.data.DataLoader, etc. The code also uses numpy to generate and convert the datasets, and torch.tensor to convert them to PyTorch tensors.

I chose the parameters used based on the paper Machine Learning for Conservative-to-Primitive in Relativistic Hydrodynamics by Dieseldorst et al. Specifically,

* The number of neurons in the first hidden layer (600) and the second hidden layer (200) are chosen according to the paper, which states that these values give good results for the neural network.
* The activation functions (sigmoid and ReLU) are chosen according to the paper, which states that sigmoid functions are used for the hidden layers and ReLU is used for the output layer.
* The initial learning rate (6e-4) is chosen according to the paper, which states that this value is used for the Adam optimizer.
* The factor to reduce the learning rate by (0.5) is chosen according to the paper, which states that this value is used for the learning rate scheduler.
* The number of epochs to wait before reducing the learning rate (10) is chosen according to the paper, which states that this value is used for the learning rate scheduler.
* The threshold for improvement in loss over the last five epochs (0.0005) is chosen according to the paper, which states that this value is used for the learning rate scheduler.
* The size of the mini-batches (32) is chosen according to the paper, which states that this value is used for training the neural network.
* The size of the training dataset (80000) and the test dataset (10000) are chosen according to the paper, which states that these values are used for generating and evaluating the neural network.
* The ranges for sampling the primitive variables (rho, e, vx) are chosen according to the paper, which states that these values are used for generating the datasets.

In [1]:
# Importing PyTorch and other libraries
import torch # PyTorch is a library for deep learning and tensor computation
import torch.nn as nn # nn is a module that provides classes and functions for building neural networks
import torch.optim as optim # optim is a module that provides optimization algorithms for neural networks
import numpy as np # numpy is a library for scientific computing and array manipulation

# Setting the random seed for reproducibility
torch.manual_seed(0) # This sets the random seed for PyTorch's random number generator
np.random.seed(0) # This sets the random seed for numpy's random number generator

# Defining the constants
GAMMA = 5/3 # The adiabatic index for the Γ-law equation of state
N = 80000 # The size of the training dataset
M = 10000 # The size of the test dataset
BATCH_SIZE = 32 # The size of the mini-batches
LR = 6e-4 # The initial learning rate
LR_FACTOR = 0.5 # The factor to reduce the learning rate by
LR_PATIENCE = 10 # The number of epochs to wait before reducing the learning rate
LR_THRESHOLD = 0.0005 # The threshold for improvement in loss over the last five epochs

# Defining the ranges for sampling the primitive variables
RHO_MIN = 0.0 # The minimum value for density
RHO_MAX = 10.1 # The maximum value for density
E_MIN = 0.0 # The minimum value for specific internal energy
E_MAX = 2.02 # The maximum value for specific internal energy
VX_MIN = -0.721 # The minimum value for velocity in x direction
VX_MAX = 0.721 # The maximum value for velocity in x direction


In [2]:
# Defining the functions to convert between primitive and conservative variables
def prim_to_cons(rho, e, vx):
    """
    Converts primitive variables (density, specific internal energy, velocity) to conservative variables (density, momentum, energy density)
    """
    w = 1 / np.sqrt(1 - vx**2) # Lorentz factor
    # TODO: Check this equation.
    #h = 1 + e + (GAMMA - 1) * rho * e / (GAMMA * rho) # Specific enthalpy
    h = 1 + e + (GAMMA - 1) * rho * e / rho # Specific enthalpy
    d = rho * w # Conserved density
    sx = rho * h * w**2 * vx # Conserved momentum in x direction
    tau = rho * h * w**2 - (GAMMA - 1) * rho * e - d # Conserved energy density
    return d, sx, tau

def cons_to_prim(d, sx, tau):
    """
    Converts conservative variables (density, momentum, energy density) to primitive variables (density, specific internal energy, velocity)
    """
    vx = sx / (tau + d + p) # Primitive velocity in x direction
    w = 1 / np.sqrt(1 - vx**2) # Lorentz factor
    e = (tau + d * (1 - w) + p * (1 - w**2)) / (d * w) # Primitive specific internal energy
    rho = d / w # Primitive density
    return rho, e, vx


In [3]:

# Generating the training and test datasets by sampling the primitive variables and converting them to conservative variables
rho_train = np.random.uniform(RHO_MIN, RHO_MAX, N) # Sampling density for training dataset from a uniform distribution over the specified range
e_train = np.random.uniform(E_MIN, E_MAX, N) # Sampling specific internal energy for training dataset from a uniform distribution over the specified range
vx_train = np.random.uniform(VX_MIN, VX_MAX, N) # Sampling velocity in x direction for training dataset from a uniform distribution over the specified range

d_train, sx_train, tau_train = prim_to_cons(rho_train, e_train, vx_train) # Converting primitive variables to conservative variables for training dataset using the defined function

rho_test = np.random.uniform(RHO_MIN, RHO_MAX, M) # Sampling density for test dataset from a uniform distribution over the specified range
e_test = np.random.uniform(E_MIN, E_MAX, M) # Sampling specific internal energy for test dataset from a uniform distribution over the specified range
vx_test = np.random.uniform(VX_MIN, VX_MAX, M) # Sampling velocity in x direction for test dataset from a uniform distribution over the specified range

d_test, sx_test, tau_test = prim_to_cons(rho_test, e_test, vx_test) # Converting primitive variables to conservative variables for test dataset using the defined function

# Converting the datasets to PyTorch tensors and normalizing them by dividing by their maximum values
d_max = max(d_train.max(), d_test.max()) # Maximum value of conserved density
sx_max = max(sx_train.max(), sx_test.max()) # Maximum value of conserved momentum in x direction
tau_max = max(tau_train.max(), tau_test.max()) # Maximum value of conserved energy density

d_train_tensor = torch.tensor(d_train / d_max, dtype=torch.float32) # Converting and normalizing conserved density for training dataset
sx_train_tensor = torch.tensor(sx_train / sx_max, dtype=torch.float32) # Converting and normalizing conserved momentum in x direction for training dataset
tau_train_tensor = torch.tensor(tau_train / tau_max, dtype=torch.float32) # Converting and normalizing conserved energy density for training dataset
p_train_tensor = torch.tensor((GAMMA - 1) * rho_train * e_train, dtype=torch.float32) # Converting pressure for training dataset

d_test_tensor = torch.tensor(d_test / d_max, dtype=torch.float32) # Converting and normalizing conserved density for test dataset
sx_test_tensor = torch.tensor(sx_test / sx_max, dtype=torch.float32) # Converting and normalizing conserved momentum in x direction for test dataset
tau_test_tensor = torch.tensor(tau_test / tau_max, dtype=torch.float32) # Converting and normalizing conserved energy density for test dataset
p_test_tensor = torch.tensor((GAMMA - 1) * rho_test * e_test, dtype=torch.float32) # Converting pressure for test dataset


In [4]:
# Defining the neural network architecture
class Net(nn.Module):
    """
    A fully connected feedforward neural network with two hidden layers and sigmoid activation functions
    """
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(3, 600) # The first hidden layer with 600 neurons and 3 inputs (conservative variables)
        self.fc2 = nn.Linear(600, 200) # The second hidden layer with 200 neurons and 600 inputs (outputs of the first hidden layer)
        self.fc3 = nn.Linear(200, 1) # The output layer with one neuron and 200 inputs (outputs of the second hidden layer)
        self.sigmoid = nn.Sigmoid() # The sigmoid activation function
        self.relu = nn.ReLU() # The ReLU nonlinearity applied to the output

    def forward(self, x):
        x = self.sigmoid(self.fc1(x)) # The first hidden layer with sigmoid activation
        x = self.sigmoid(self.fc2(x)) # The second hidden layer with sigmoid activation
        x = self.relu(self.fc3(x)) # The output layer with ReLU nonlinearity
        return x


In [5]:

# Creating an instance of the neural network
net = Net()

# Defining the loss function as mean squared error
criterion = nn.MSELoss()

# Defining the optimizer as Adam with the initial learning rate
optimizer = optim.Adam(net.parameters(), lr=LR)

# Defining a scheduler to reduce the learning rate by a factor when the loss does not improve by a threshold over a patience period
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=LR_FACTOR, patience=LR_PATIENCE, threshold=LR_THRESHOLD)


In [6]:

# Defining a function to calculate the L1 and L_inf norms of the errors on a dataset
def calculate_errors(data_loader):
    """
    Calculates the L1 and L_inf norms of the errors on a dataset given by a data loader
    """
    l1_error = 0.0 # The sum of absolute errors
    l_inf_error = 0.0 # The maximum absolute error
    n_samples = 0 # The number of samples in the dataset

    with torch.no_grad(): # No need to track gradients for evaluation
        for inputs, labels in data_loader: # Loop over the mini-batches in the data loader
            outputs = net(inputs) # Forward pass of the neural network
            errors = torch.abs(outputs - labels) # Absolute errors between outputs and labels
            l1_error += errors.sum().item() # Update the sum of absolute errors
            l_inf_error = max(l_inf_error, errors.max().item()) # Update the maximum absolute error
            n_samples += inputs.size(0) # Update the number of samples

    l1_error /= n_samples # Calculate the average absolute error (L1 norm)
    return l1_error, l_inf_error


In [7]:

# Creating data loaders for the training and test datasets with random shuffling and mini-batch size
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.cat((d_train_tensor.unsqueeze(1), sx_train_tensor.unsqueeze(1), tau_train_tensor.unsqueeze(1)), dim=1), p_train_tensor.unsqueeze(1)), batch_size=BATCH_SIZE, shuffle=True) # Creating a data loader for the training dataset by concatenating the input tensors and the label tensor along the second dimension and creating a tensor dataset, then specifying the batch size and shuffling option
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.cat((d_test_tensor.unsqueeze(1), sx_test_tensor.unsqueeze(1), tau_test_tensor.unsqueeze(1)), dim=1), p_test_tensor.unsqueeze(1)), batch_size=BATCH_SIZE, shuffle=True) # Creating a data loader for the test dataset by concatenating the input tensors and the label tensor along the second dimension and creating a tensor dataset, then specifying the batch size and shuffling option

# Training the neural network
for epoch in range(100): # Loop over 100 epochs
    running_loss = 0.0 # The running loss for the training dataset
    for i, data in enumerate(train_loader, 0): # Loop over the mini-batches in the training data loader
        inputs, labels = data # Get the inputs and labels from the mini-batch
        optimizer.zero_grad() # Zero the parameter gradients
        outputs = net(inputs) # Forward pass of the neural network
        loss = criterion(outputs, labels) # Calculate the loss
        loss.backward() # Backward pass to calculate the gradients
        optimizer.step() # Update the parameters with gradient descent
        running_loss += loss.item() # Update the running loss

    train_loss = running_loss / len(train_loader) # Calculate the average loss for the training dataset
    scheduler.step(train_loss) # Adjust the learning rate based on the training loss
    train_l1_error, train_l_inf_error = calculate_errors(train_loader) # Calculate the L1 and L_inf norms of the errors for the training dataset
    test_l1_error, test_l_inf_error = calculate_errors(test_loader) # Calculate the L1 and L_inf norms of the errors for the test dataset

    print(f'Epoch {epoch + 1}, Train Loss: {train_loss:.4f}, Train L1 Error: {train_l1_error:.4f}, Train L_inf Error: {train_l_inf_error:.4f}, Test L1 Error: {test_l1_error:.4f}, Test L_inf Error: {test_l_inf_error:.4f}') # Print the results for the epoch

print('Finished Training') # Print a message when training is done


Epoch 1, Train Loss: 20.5337, Train L1 Error: 3.3998, Train L_inf Error: 13.5472, Test L1 Error: 3.3241, Test L_inf Error: 13.2536
Epoch 2, Train Loss: 20.5337, Train L1 Error: 3.3998, Train L_inf Error: 13.5472, Test L1 Error: 3.3241, Test L_inf Error: 13.2536


KeyboardInterrupt: 

In [1]:
%%script echo skipping

# Importing PyTorch and other libraries
import torch # PyTorch is a library for deep learning and tensor computation
import torch.nn as nn # nn is a module that provides classes and functions for building neural networks
import torch.optim as optim # optim is a module that provides optimization algorithms for neural networks
import numpy as np # numpy is a library for scientific computing and array manipulation

# Setting the random seed for reproducibility
torch.manual_seed(0) # This sets the random seed for PyTorch's random number generator
np.random.seed(0) # This sets the random seed for numpy's random number generator

# Defining the constants
GAMMA = 5/3 # The adiabatic index for the Γ-law equation of state
N = 80000 # The size of the training dataset
M = 10000 # The size of the test dataset
BATCH_SIZE = 32 # The size of the mini-batches
LR = 6e-4 # The initial learning rate
LR_FACTOR = 0.5 # The factor to reduce the learning rate by
LR_PATIENCE = 10 # The number of epochs to wait before reducing the learning rate
LR_THRESHOLD = 0.0005 # The threshold for improvement in loss over the last five epochs

# Defining the ranges for sampling the primitive variables
RHO_MIN = 0.0 # The minimum value for density
RHO_MAX = 10.1 # The maximum value for density
E_MIN = 0.0 # The minimum value for specific internal energy
E_MAX = 2.02 # The maximum value for specific internal energy
VX_MIN = -0.721 # The minimum value for velocity in x direction
VX_MAX = 0.721 # The maximum value for velocity in x direction

# Defining the functions to convert between primitive and conservative variables
def prim_to_cons(rho, e, vx):
    """
    Converts primitive variables (density, specific internal energy, velocity) to conservative variables (density, momentum, energy density)
    """
    w = 1 / np.sqrt(1 - vx**2) # Lorentz factor
    # TODO: Check this equation.
    h = 1 + e + (GAMMA - 1) * rho * e / (GAMMA * rho) # Specific enthalpy
    d = rho * w # Conserved density
    sx = rho * h * w**2 * vx # Conserved momentum in x direction
    tau = rho * h * w**2 - (GAMMA - 1) * rho * e - d # Conserved energy density
    return d, sx, tau

def cons_to_prim(d, sx, tau):
    """
    Converts conservative variables (density, momentum, energy density) to primitive variables (density, specific internal energy, velocity)
    """
    vx = sx / (tau + d + p) # Primitive velocity in x direction
    w = 1 / np.sqrt(1 - vx**2) # Lorentz factor
    e = (tau + d * (1 - w) + p * (1 - w**2)) / (d * w) # Primitive specific internal energy
    rho = d / w # Primitive density
    return rho, e, vx

# Generating the training and test datasets by sampling the primitive variables and converting them to conservative variables
rho_train = np.random.uniform(RHO_MIN, RHO_MAX, N) # Sampling density for training dataset from a uniform distribution over the specified range
e_train = np.random.uniform(E_MIN, E_MAX, N) # Sampling specific internal energy for training dataset from a uniform distribution over the specified range
vx_train = np.random.uniform(VX_MIN, VX_MAX, N) # Sampling velocity in x direction for training dataset from a uniform distribution over the specified range

d_train, sx_train, tau_train = prim_to_cons(rho_train, e_train, vx_train) # Converting primitive variables to conservative variables for training dataset using the defined function

rho_test = np.random.uniform(RHO_MIN, RHO_MAX, M) # Sampling density for test dataset from a uniform distribution over the specified range
e_test = np.random.uniform(E_MIN, E_MAX, M) # Sampling specific internal energy for test dataset from a uniform distribution over the specified range
vx_test = np.random.uniform(VX_MIN, VX_MAX, M) # Sampling velocity in x direction for test dataset from a uniform distribution over the specified range

d_test, sx_test, tau_test = prim_to_cons(rho_test, e_test, vx_test) # Converting primitive variables to conservative variables for test dataset using the defined function

# Converting the datasets to PyTorch tensors and normalizing them by dividing by their maximum values
d_max = max(d_train.max(), d_test.max()) # Maximum value of conserved density
sx_max = max(sx_train.max(), sx_test.max()) # Maximum value of conserved momentum in x direction
tau_max = max(tau_train.max(), tau_test.max()) # Maximum value of conserved energy density

d_train_tensor = torch.tensor(d_train / d_max, dtype=torch.float32) # Converting and normalizing conserved density for training dataset
sx_train_tensor = torch.tensor(sx_train / sx_max, dtype=torch.float32) # Converting and normalizing conserved momentum in x direction for training dataset
tau_train_tensor = torch.tensor(tau_train / tau_max, dtype=torch.float32) # Converting and normalizing conserved energy density for training dataset
p_train_tensor = torch.tensor((GAMMA - 1) * rho_train * e_train, dtype=torch.float32) # Converting pressure for training dataset

d_test_tensor = torch.tensor(d_test / d_max, dtype=torch.float32) # Converting and normalizing conserved density for test dataset
sx_test_tensor = torch.tensor(sx_test / sx_max, dtype=torch.float32) # Converting and normalizing conserved momentum in x direction for test dataset
tau_test_tensor = torch.tensor(tau_test / tau_max, dtype=torch.float32) # Converting and normalizing conserved energy density for test dataset
p_test_tensor = torch.tensor((GAMMA - 1) * rho_test * e_test, dtype=torch.float32) # Converting pressure for test dataset

# Defining the neural network architecture
class Net(nn.Module):
    """
    A fully connected feedforward neural network with two hidden layers and sigmoid activation functions
    """
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(3, 600) # The first hidden layer with 600 neurons and 3 inputs (conservative variables)
        self.fc2 = nn.Linear(600, 200) # The second hidden layer with 200 neurons and 600 inputs (outputs of the first hidden layer)
        self.fc3 = nn.Linear(200, 1) # The output layer with one neuron and 200 inputs (outputs of the second hidden layer)
        self.sigmoid = nn.Sigmoid() # The sigmoid activation function
        self.relu = nn.ReLU() # The ReLU nonlinearity applied to the output

    def forward(self, x):
        x = self.sigmoid(self.fc1(x)) # The first hidden layer with sigmoid activation
        x = self.sigmoid(self.fc2(x)) # The second hidden layer with sigmoid activation
        x = self.relu(self.fc3(x)) # The output layer with ReLU nonlinearity
        return x

# Creating an instance of the neural network
net = Net()

# Defining the loss function as mean squared error
criterion = nn.MSELoss()

# Defining the optimizer as Adam with the initial learning rate
optimizer = optim.Adam(net.parameters(), lr=LR)

# Defining a scheduler to reduce the learning rate by a factor when the loss does not improve by a threshold over a patience period
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=LR_FACTOR, patience=LR_PATIENCE, threshold=LR_THRESHOLD)

# Defining a function to calculate the L1 and L_inf norms of the errors on a dataset
def calculate_errors(data_loader):
    """
    Calculates the L1 and L_inf norms of the errors on a dataset given by a data loader
    """
    l1_error = 0.0 # The sum of absolute errors
    l_inf_error = 0.0 # The maximum absolute error
    n_samples = 0 # The number of samples in the dataset

    with torch.no_grad(): # No need to track gradients for evaluation
        for inputs, labels in data_loader: # Loop over the mini-batches in the data loader
            outputs = net(inputs) # Forward pass of the neural network
            errors = torch.abs(outputs - labels) # Absolute errors between outputs and labels
            l1_error += errors.sum().item() # Update the sum of absolute errors
            l_inf_error = max(l_inf_error, errors.max().item()) # Update the maximum absolute error
            n_samples += inputs.size(0) # Update the number of samples

    l1_error /= n_samples # Calculate the average absolute error (L1 norm)
    return l1_error, l_inf_error

# Creating data loaders for the training and test datasets with random shuffling and mini-batch size
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.cat((d_train_tensor.unsqueeze(1), sx_train_tensor.unsqueeze(1), tau_train_tensor.unsqueeze(1)), dim=1), p_train_tensor.unsqueeze(1)), batch_size=BATCH_SIZE, shuffle=True) # Creating a data loader for the training dataset by concatenating the input tensors and the label tensor along the second dimension and creating a tensor dataset, then specifying the batch size and shuffling option
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.cat((d_test_tensor.unsqueeze(1), sx_test_tensor.unsqueeze(1), tau_test_tensor.unsqueeze(1)), dim=1), p_test_tensor.unsqueeze(1)), batch_size=BATCH_SIZE, shuffle=True) # Creating a data loader for the test dataset by concatenating the input tensors and the label tensor along the second dimension and creating a tensor dataset, then specifying the batch size and shuffling option

# Training the neural network
for epoch in range(100): # Loop over 100 epochs
    running_loss = 0.0 # The running loss for the training dataset
    for i, data in enumerate(train_loader, 0): # Loop over the mini-batches in the training data loader
        inputs, labels = data # Get the inputs and labels from the mini-batch
        optimizer.zero_grad() # Zero the parameter gradients
        outputs = net(inputs) # Forward pass of the neural network
        loss = criterion(outputs, labels) # Calculate the loss
        loss.backward() # Backward pass to calculate the gradients
        optimizer.step() # Update the parameters with gradient descent
        running_loss += loss.item() # Update the running loss

    train_loss = running_loss / len(train_loader) # Calculate the average loss for the training dataset
    scheduler.step(train_loss) # Adjust the learning rate based on the training loss
    train_l1_error, train_l_inf_error = calculate_errors(train_loader) # Calculate the L1 and L_inf norms of the errors for the training dataset
    test_l1_error, test_l_inf_error = calculate_errors(test_loader) # Calculate the L1 and L_inf norms of the errors for the test dataset

    print(f'Epoch {epoch + 1}, Train Loss: {train_loss:.4f}, Train L1 Error: {train_l1_error:.4f}, Train L_inf Error: {train_l_inf_error:.4f}, Test L1 Error: {test_l1_error:.4f}, Test L_inf Error: {test_l_inf_error:.4f}') # Print the results for the epoch

print('Finished Training') # Print a message when training is done


Epoch 1, Train Loss: 20.5337, Train L1 Error: 3.3998, Train L_inf Error: 13.5472, Test L1 Error: 3.3241, Test L_inf Error: 13.2536
Epoch 2, Train Loss: 20.5337, Train L1 Error: 3.3998, Train L_inf Error: 13.5472, Test L1 Error: 3.3241, Test L_inf Error: 13.2536
Epoch 3, Train Loss: 20.5337, Train L1 Error: 3.3998, Train L_inf Error: 13.5472, Test L1 Error: 3.3241, Test L_inf Error: 13.2536
Epoch 4, Train Loss: 20.5337, Train L1 Error: 3.3998, Train L_inf Error: 13.5472, Test L1 Error: 3.3241, Test L_inf Error: 13.2536
Epoch 5, Train Loss: 20.5337, Train L1 Error: 3.3998, Train L_inf Error: 13.5472, Test L1 Error: 3.3241, Test L_inf Error: 13.2536
Epoch 6, Train Loss: 20.5337, Train L1 Error: 3.3998, Train L_inf Error: 13.5472, Test L1 Error: 3.3241, Test L_inf Error: 13.2536
Epoch 7, Train Loss: 20.5337, Train L1 Error: 3.3998, Train L_inf Error: 13.5472, Test L1 Error: 3.3241, Test L_inf Error: 13.2536
Epoch 8, Train Loss: 20.5337, Train L1 Error: 3.3998, Train L_inf Error: 13.5472, T

KeyboardInterrupt: 